In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\ikbal\\Desktop\\projects\\flower-classification'

In [2]:
import tensorflow as tf

# model = tf.keras.applications.MobileNetV2(
#             input_shape=[224, 224, 3],
#             weights="imagenet",
#             include_top=False
#         )

# for layer in model.layers:
#     layer.trainable = False

# flatten_in = tf.keras.layers.Flatten()(model.output)
# prediction = tf.keras.layers.Dense(
#     units=17,
#     activation="softmax"
# )(flatten_in)

# full_model = tf.keras.models.Model(
#     inputs=model.input,
#     outputs=prediction
# )

# full_model.compile(
#     optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=["accuracy"]
# )

# full_model.summary()


In [3]:
import tensorflow as tf

# Define the 3-layer CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=5, activation='softmax')  # Assuming 17 classes
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [4]:
[224, 224, 3][: -1]

[224, 224]

In [5]:
training_data = os.path.join("artifacts/data_ingestion", "flower_images")

datagenerator_kwargs = dict(
    rescale = 1./255,
    validation_split=0.20
)

dataflow_kwargs = dict(
    target_size=[224, 224, 3][:-1],
    batch_size=16,
    interpolation="bilinear"
)

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
)

valid_generator = valid_datagenerator.flow_from_directory(
    directory=training_data,
    subset="validation",
    shuffle=False,
    **dataflow_kwargs
)

train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    **datagenerator_kwargs
)


train_generator = train_datagenerator.flow_from_directory(
    directory=training_data,
    subset="training",
    shuffle=True,
    **dataflow_kwargs
)

Found 1000 images belonging to 5 classes.
Found 4000 images belonging to 5 classes.


In [6]:
reverse_dict = {v: k for k, v in train_generator.class_indices.items()} 
reverse_dict

{0: 'Lilly', 1: 'Lotus', 2: 'Orchid', 3: 'Sunflower', 4: 'Tulip'}

In [7]:
train_generator.labels

array([0, 0, 0, ..., 4, 4, 4])

In [8]:
if tf.test.gpu_device_name():
    print('GPU found')
    device = '/device:GPU:0'  # Use the first GPU
else:
    print('No GPU found')
    device = '/device:CPU:0'

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
with tf.device(device):
    model.fit(
        train_generator,
        epochs=5,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        validation_data=valid_generator,
    )


No GPU found
Epoch 1/5
250/250 [==============================] - 654s 3s/step - loss: 1.4360 - accuracy: 0.3725 - val_loss: 1.2774 - val_accuracy: 0.4748
Epoch 2/5
161/250 [==================>...........] - ETA: 2:34 - loss: 1.2974 - accuracy: 0.4488

: 

In [51]:
from tensorflow.keras.preprocessing import image
import numpy as np

imagename = "artifacts/data_ingestion/flowe_images/Daffodil/image_0006.jpg"
test_image = image.load_img(imagename, target_size = (224,224, 3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = np.argmax(full_model.predict(test_image), axis=1)
print(reverse_dict[result[0]])

1/1 [==============================] - 0s 68ms/step
Crocus


In [40]:
full_model